# Comment créer une base de données vectorielle avec PostgreSLQ et pgvector

## Etapes

1. Créer la BDD PostgreSQL + pgvector dans un conteneur Docker
2. Se connecter avec psql (pour vérification)
3. Importer les données (textes)
4. Créer les vecteurs avec Sentence Transformers
5. Créer une table 'quora' dans la BDD
6. Sauvegarder les données et les vecteurs dans la BDD
7. Utiliser la recherche vectorielle 

## 1. Docker : Créer la BDD PostgreSQL + pgvector avec dans un conteneur 

### Docker-compose.yaml

Tout d'abord, nous devons créer un fichier `docker-compose.yml` avec les services nécessaires.

Dans ce fichier, nous définissons un service appelé `db` qui est basé sur l'image Docker `pgvector/pgvector:pg16`. Le service expose le port `5432` pour interagir avec la base de données et configure des variables d'environnement pour le nom de la base de données, l'utilisateur, le mot de passe et la méthode d'authentification. De plus, nous montons un fichier `init.sql` dans le répertoire `/docker-entrypoint-initdb.d` à l'intérieur du conteneur à des fins d'initialisation.

### Fichier init.sql

Dans ce script `init.sql`, nous activons l'extension `pgvector`, si elle n'existe pas déjà. Ensuite, nous créons une table appelée `embedding` avec les colonnes : `id`, `embedding`, `text` et `created_at`.

### Créer le conteneur

```
docker-compose up -d
```

Cette commande créera un conteneur Docker avec le serveur PostgreSQL et l'extension `pgvector` déjà installés et configurés, en fonction des spécifications du fichier `docker-compose.yml`.


#### Obsolète 

Créer un base de données PostgreSQL avec `docker run`

```
docker run -d --name postgresCont -p 5432:5432 -e POSTGRES_PASSWORD=pass123 postgres
docker run -d --name postgresCont -p 5432:5432 -e POSTGRES_PASSWORD=pass123 pgvector/pgvector:pg16
docker exec -it postgresCont bash
psql -h localhost -U postgres
```

## 2. Se connecter avec psql (dans le conteneur)

### Aller dans le conteneur

`docker exec -it <container id> bash`

### Se conncter à la base de donnée avec `psql`

`psql -h localhost -U postgres -d vectordb`

In [ ]:
# ! pip install sentence-transformers "psycopg[binary]" --quiet

In [13]:
import os
import time

import pandas as pd
import psycopg
from sentence_transformers import SentenceTransformer

In [23]:
# Modèle Sentence Transformer pour l'encodage du jeu de données ou de la requète de recherche sémantique
# model_name = 'all-MiniLM-L6-v2' # Vector size = 384
# model_name = "BAAI/bge-m3" # vector size = 1024
model_name = "dangvantuan/sentence-camembert-base" # Pre-trained sentence embedding for French.
model = SentenceTransformer(model_name, device='cuda')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

**SentenceTransformers** est un framework Python pour la création de représentations vectorielles de phrases, de textes et d'images.

- https://www.sbert.net/index.html
- https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
- https://huggingface.co/BAAI/bge-m3
- https://huggingface.co/dangvantuan/sentence-camembert-large

## 3. Importer le jeu de données (textes)

Le jeu de donnée consiste en **11812 paires de question-réponses**, extraites du dataset **"piaf v1.2"**, disponible sur https://www.data.gouv.fr/. L'objectif est de créer des représentations vectorielles pour les questions avec un modèle **Sentence Transformers (sBert)**. Ainsi, nous pourrons effectuer une **recherche sémantique** dans les questions depuis une requète textuelle ou **trouver les questions les plus similaires** entre elles.

|  | Description |
| --- | --- |
| Nom du fichier | question-reponse-sans-texte.csv |
| Description du fichier | Extraction des questions-réponses à partir du fichier piaf-v1.2.json pour les visualiser au format CSV |
| Colonnes | 2 colonnes: ['question', 'reponse'] |
| Lignes | 11812 lignes | 
| Type MIME | text/csv |
| Créée le | 25 mars 2021 |
| Taille | 990.9Ko |

- https://www.data.gouv.fr/fr/datasets/piaf-le-dataset-francophone-de-questions-reponses/

In [17]:
# question-reponse-sans-texte.csv
dataset_url = "https://www.data.gouv.fr/fr/datasets/r/14159082-d1be-417e-a67c-c3c494c7a4ad"

dataset_df = pd.read_csv(dataset_url)

In [18]:
dataset_df.shape

(11812, 2)

In [19]:
corpus_questions = dataset_df.loc[:, 'question'].tolist()
corpus_reponses = dataset_df.loc[:, 'reponse'].tolist()

In [22]:
# Afficher les 10 premières paires de question-réponse.
for question, reponse in zip(corpus_questions[:10], corpus_reponses[:10]):
    print(f"{question} => {reponse}")

Quel architecte fût à l'origine des plans du Woolworth building? => Cass Gilbert
Où se trouvait Franck Woolworth lors de l'inauguration de son immeuble New Yorkais ? => Washington
Comment fût payé le bâtiment commandé par Franck Woolworth? => en cash
En quelle année ouvrit le Woolworth Building ? => 1913
Qui commanda la construction du Woolworth Building ? => Frank Woolworth
Quelle femme devint reine aux côtés de Philippe le Bel ? => Jeanne Ire de Navarre
Quel créancier du roi fut supprimé en 1312 ? => l'ordre du Temple
Quel créancier du roi fut supprimé en 1312 ? => l'ordre du Temple
Quelle raison pousse Philippe Le Bel à organiser les premiers Etats généraux ? => pour lever de nouveaux impôts
Quel souverain utilise les dévaluations monétaires pour s'enrichir ? => Philippe IV le Bel


## 4. Encoder les questions avec Sentence Transformers

In [ ]:
embeddings_questions = model.encode(corpus_questions, show_progress_bar=True).tolist()

## 5. Créer une table 'piaf' dans la BDD

### Connexion à la base de données avec Psycopg

- https://www.psycopg.org/psycopg3/docs/basic/usage.html

In [8]:
# DEFINE THE DATABASE CREDENTIALS
user = 'testuser'
password = 'testpwd'
host = 'localhost'
port = 5432
database = 'vectordb'

db_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"

### Effacer la table pour éviter d'ajouter des lignes en double

In [ ]:
# Connect to an existing database
with psycopg.connect(conninfo=db_url) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        # Execute a command: this creates a new table
        cur.execute("""
            DROP TABLE piaf; 
            """)
        # Make the changes to the database persistent
        conn.commit()

### Création de la table

In [99]:
# Connect to an existing database
with psycopg.connect(conninfo=db_url) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        # Execute a command: this creates a new table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS piaf (
                id serial PRIMARY KEY,
                question text NOT NULL,
                reponse text NOT NULL,
                embedding vector(384) NOT NULL,
                created_at timestamptz DEFAULT now()
                );
            """)
        # Make the changes to the database persistent
        conn.commit()

## 6. Sauvegarder les données et les vecteurs dans la BDD

In [100]:
with psycopg.connect(conninfo=db_url) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        # Execute a command
        for question, reponse, embedding in zip(corpus_questions, corpus_reponses, embeddings_questions):
            cur.execute("""INSERT INTO piaf (question, reponse, embedding) VALUES (%s, %s, %s);""", (question, reponse, embedding))
        # Make the changes to the database persistent
        conn.commit()

## 7. Utiliser la recherche vectorielle 

- https://huggingface.co/sentence-transformers/quora-distilbert-multilingual

### Rechercher par requète textuelle

In [6]:
%%time
query = "Quelle est la date de la révolution ?"

# Encoder la requète avec Sentence Transformers
embedding_query = model.encode(query, show_progress_bar=True).tolist()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 720 ms, sys: 132 ms, total: 853 ms
Wall time: 1.3 s


In [9]:
%%time
# Connect to an existing database
with psycopg.connect(conninfo=db_url) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        # Execute a command: this creates a new table
        res = cur.execute("""
            SELECT
                id,
                1 - (embedding <=> %s) AS cosine_similarity,
                question,
                reponse
            FROM piaf ORDER BY cosine_similarity DESC LIMIT 10;
        """, (str(embedding_query), )).fetchall()
        for row in res:
            print(f"id : {row[0]} | score : {round(row[1], 4)} | Question : {row[2]} | Réponse : {row[3]}")

id : 790 | score : 0.8086 | Question : Quelle est la conséquence de la Révolution de 1911 ? | Réponse : la proclamation de la république de Chine
id : 3835 | score : 0.7912 | Question : Quel est un des éléments de l'avènement de la révolution ? | Réponse : la baisse des salaires réels
id : 4156 | score : 0.7891 | Question : quelle est la date de la réforme? | Réponse : 1995
id : 4155 | score : 0.7891 | Question : quelle est la date de la réforme? | Réponse : 1995
id : 7495 | score : 0.7807 | Question : Quelle est la date de la libération de Paris ? | Réponse : 25 août 1944
id : 3762 | score : 0.7576 | Question : Quelle est la date de l’inauguration ? | Réponse : 11 octobre 1854
id : 3763 | score : 0.7576 | Question : Quelle est la date de l’inauguration ? | Réponse : le 11 octobre 1854
id : 7246 | score : 0.7532 | Question : De quand date la constitution ? | Réponse : 1974
id : 11371 | score : 0.7186 | Question : Quelle est la date exacte du meurtre qui déclencha la guerre ? | Réponse 

In [10]:
# Retourner les résultats dans une DataFrame
def query_to_dataframe(query, column_names):

    # Connect to an existing database
    with psycopg.connect(conninfo=db_url) as conn:
        # Open a cursor to perform database operations
        with conn.cursor() as cur:
            embeddings_query = model.encode(query).tolist()
            # Execute a command: this creates a new table
            res = cur.execute("""
                SELECT
                    id,
                    1 - (embedding <=> %s) AS cosine_similarity,
                    question,
                    reponse
                FROM piaf ORDER BY cosine_similarity DESC LIMIT 10;
            """, (str(embeddings_query), )).fetchall()
    res_df = pd.DataFrame(res, columns=column_names)
    return res_df

In [11]:
%%time
# Cherchez une question en changeant cette requète
query = "Quelle est la date de la libération ?"
query_to_dataframe(query, ['id', 'cosine_similarity', 'question', 'reponse'])

CPU times: user 17.4 ms, sys: 9.44 ms, total: 26.9 ms
Wall time: 44.5 ms


,id,cosine_similarity,question,reponse
0,7495,0.898055,Quelle est la date de la libération de Paris ?,25 août 1944
1,3763,0.759356,Quelle est la date de l’inauguration ?,le 11 octobre 1854
2,3762,0.759356,Quelle est la date de l’inauguration ?,11 octobre 1854
3,4155,0.755041,quelle est la date de la réforme?,1995
4,4156,0.755041,quelle est la date de la réforme?,1995
5,7810,0.748075,De quand date le début de la domination angevine?,1301
6,1019,0.746150,Quelle est la date de la fin de la tournée sec...,le 10 juin 2007
7,4256,0.738092,Quelle est la date de la reddition de la Grand...,juillet 1857
8,4257,0.738092,Quelle est la date de la reddition de la Grand...,juillet 1857
9,6857,0.726841,quelle est la date de naissance de cézanne?,19 janvier 1839


### Les plus proches voisins d'une ligne (id aléatoire)

Résultats : 
- La première ligne est une ligne choisie au hasard dans la base de donnée.
- Les lignes suivantes sont les lignes les plus proches sémantiquement, par ordre décroissant de similarité cosinus.

In [12]:
import random
rand_id = random.randint(0,11812)

print(f"Random id : {rand_id}")
print()

# Connect to an existing database
with psycopg.connect(conninfo=db_url) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        res = cur.execute("""
            SELECT
            id,
            1 - (embedding <=> (SELECT embedding FROM piaf WHERE id = %s)) AS cosine_similarity,
            question,
            reponse          
            FROM piaf
            ORDER BY cosine_similarity DESC LIMIT 20;
        """, (rand_id, )).fetchall()

        res_df = pd.DataFrame(res, columns=['id', 'cosine_similarity', 'question', 'reponse'])
res_df

Random id : 9221



,id,cosine_similarity,question,reponse
0,9221,1.000000,Comment meurt l'ami de Dagen ?,pendu
1,9261,0.622928,Qui meurt ?,Dina
2,3224,0.618013,Où est l'Ouganda ?,la région des Grands Lacs
3,3225,0.618013,Où est l'Ouganda ?,région des Grands Lacs
4,1080,0.598873,Qu'est-ce que l'Ouadia ?,un droit de passage
5,9253,0.598135,Que doit soulever Dagen ?,une très lourde charge
6,4060,0.592176,Comment est morte Hegwige ?,tuée lors d'une attaque de Mangemorts
7,10952,0.591812,Qui est le Dr Oudot ?,le médecin de l'expédition
8,3293,0.590706,À qui est alliée la maison d'Este ?,Carlo Gesualdo
9,3294,0.590706,À qui est alliée la maison d'Este ?,duché de Ferrare
